In [1]:
import numpy as np
import pandas as pd
import itertools as it
from collections import Counter
import calendar as clnd

In [2]:
data = pd.read_csv('C:\Projects_0\input_data\movie_bd_v5.xls')

In [137]:
data.describe()

## Подготовк к ответам
* создание словаря с ответами
* подготовка данный в источнике данных

In [3]:
# словарь для ответов
answeres = {}

In [4]:
# вставим в оригинальный датасет вычисляемый поле с прибылью от фильма [profit]
data['profit'] = data.revenue - data.budget

# обработаем дату. достанем из нее месяц и определим врмея года. 
# месяц находится в дате первый
data['release_month'] = data.release_date.apply(lambda d: int(d.split('/')[0]))

# находим по месяцу время года (через функцию) и втсавляем его датасет в поле [year_time]
def year_time(month):
    return ['Winter', 'Spring', 'Summer', 'Autumn', 'Winter'][month // 3]

data['year_time'] = data.release_month.apply(year_time)

# создаем копию датасета из прибыльных фильмов (в вопросах часто будет нужна такая выборка)
profitable_films = data[data.profit > 0].copy()

In [6]:
# data.sample(5)

In [7]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
 14  profit                1889 non-null   int64  
 15  release_month        

None

# А теперь, сами ответы на вопросы

### 1. У какого фильма из списка самый большой бюджет?

In [5]:
# version 1 (просто получаем строку с самым большим бюджетом и берем название фильма из нее)
answeres['1'] = data[data.budget == data.budget.max()].original_title.describe().top

In [6]:
# version 2 (сротируем по убыванию по полю "бюджет" и из первой строки берем название)
answeres['1'] = data.sort_values('budget', ascending=False).iloc[0].original_title

In [7]:
# information about answer
ttl1 = data.sort_values('budget', ascending=False).iloc[0].original_title
bdg1 = data.sort_values('budget', ascending=False).iloc[0].budget
print(f'Фильм {ttl1} стал самым дорогим проектом в истории, на создание которого ушло {bdg1:,d} долларов')

Фильм Pirates of the Caribbean: On Stranger Tides стал самым дорогим проектом в истории, на создание которого ушло 380,000,000 долларов


### 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
# version 1
answeres['2'] = data[data.runtime == data.runtime.max()].original_title.describe().top

In [9]:
# information about answer
long = data[data.runtime == data.runtime.max()].original_title.describe().top
time_l = data[data.runtime >= data.runtime.max()].runtime.max()
f'Собираясь посмотреть {long}, запаситесь кофе! Его длительность {time_l} мин - как бы не уснуть!'

'Собираясь посмотреть Gods and Generals, запаситесь кофе! Его длительность 214 мин - как бы не уснуть!'

### 3. Какой из фильмов самый короткий (в минутах)?

In [10]:
# version 1
answeres['3'] = data[data.runtime == data.runtime.min()].original_title.describe().top

In [11]:
# information about answer
short = data[data.runtime == data.runtime.min()].original_title.describe().top
time_s = data[data.runtime >= data.runtime.min()].runtime.min()
f'Милая анимация "{short}" не займет у вас много времени - всего {time_s} минуты.'

'Милая анимация "Winnie the Pooh" не займет у вас много времени - всего 63 минуты.'

### 4. Какова средняя длительность фильмов?

In [12]:
# version 1
answeres['4'] = data.runtime.mean()

In [13]:
# information about answer
f'{data.runtime.mean():.3f}'


'109.659'

### 5. Каково медианное значение длительности фильмов?

In [14]:
# version 1
answeres['5'] = data.runtime.median()

In [15]:
# information about answer
data.runtime.median()

107.0

### 6. Какой самый прибыльный фильм?

In [16]:
# version 1
answeres['6'] = data[data.profit == data.profit.max()].original_title.describe().top

In [17]:
# information about answer
p_ttl = data[data.profit == data.profit.max()].original_title.describe().top
prof = data[data.profit == data.profit.max()].profit.max()
print(f'"{p_ttl}", с рекордной прибылью {prof:,d} долларов')

"Avatar", с рекордной прибылью 2,544,505,847 долларов


### 7. Какой фильм самый убыточный? 

In [18]:
# version 1
answeres['7'] = data[data.profit == data.profit.min()].original_title.describe().top

In [19]:
# information about answer
l_ttl = data[data.profit == data.profit.min()].original_title.describe().top
lose = data[data.profit == data.profit.min()].profit.min()
f'Лавры главного неудачника Голливуда принадлежат фильму "{l_ttl}", с рекордными убытками {lose:,d}'

'Лавры главного неудачника Голливуда принадлежат фильму "The Lone Ranger", с рекордными убытками -165,710,090'

### 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [20]:
# version 1
answeres['8'] = len(data[data.profit > 0].profit)

In [21]:
# version 2 (или взять заранее отфильтрованный датасет только с прибыльными фильмами)
answeres['8'] = len(profitable_films)

In [22]:
# information about answer
len(data[data.profit > 0].profit)

1478

### 9. Какой фильм оказался самым кассовым в 2008 году?

In [23]:
# version 1
# применим два условия: год выхода = 2008 и макимум сборов в выорке по этому же году 
answeres['9'] = data[(data.release_year == 2008) & 
                     (data.revenue == data[data.release_year == 2008].revenue.max())].original_title.describe().top

In [24]:
# information about answer
ttl08 = data[(data.release_year == 2008) & 
     (data.revenue == data[data.release_year == 2008].revenue.max())].original_title.describe().top
rev = data[(data.release_year == 2008) & 
     (data.revenue == data[data.release_year == 2008].revenue.max())].revenue.max()
print(f'Фильм "{ttl08}", показал рекордные сборы {rev:,d} в 2008 году.')

Фильм "The Dark Knight", показал рекордные сборы 1,001,921,825 в 2008 году.


### 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?

In [25]:
# version 1 
# делаем фильтр датасета по периоду, далее ищем фильм с самым большим убытком, получаем его название
data_short = data[(data.release_year >= 2012) & (data.release_year <= 2014)]
answeres['10'] =  data_short[data_short.profit == data.profit.min()].original_title.describe().top

In [26]:
# information about answer
data_short = data[(data.release_year >= 2012) & (data.release_year <= 2014)]

ttl12 = data_short[data_short.profit == data.profit.min()].original_title.describe().top
los12 =data_short[data_short.profit == data.profit.min()].profit.min()
print(f'в период 2012-2014 годы фильм "{ttl12}", показал рекордные убытки {los12:,d}')

в период 2012-2014 годы фильм "The Lone Ranger", показал рекордные убытки -165,710,090


### 11. Какого жанра фильмов больше всего?

In [27]:
data11 = data.copy()
# в поле genres жанры перечислены через разделитель. Преобразуем их в список и "раскроем" функцией explode
data11 = data11.genres.str.split('|').explode().value_counts()
answeres['11'] = data11.index[0]

In [28]:
# information about answer
gnr11 = data11.values[0]
print(f'Больше всего ({gnr11}) фильмов снимается в жанре "{data11.index[0]}"')

Больше всего (782) фильмов снимается в жанре "Drama"


### 12. Фильмы какого жанра чаще всего становятся прибыльными?

In [29]:
# version 1
data12 = data[data.profit > 0].genres.str.split('|').explode().value_counts()
answeres['12'] = data12.index[0]

In [30]:
# version 2 (используем заранее отфильтрованный датасет только с прибыльными фильмами)
answeres['12'] = profitable_films.genres.str.split('|').explode().value_counts().index[0]

In [31]:
# information about answer
gnr12 = data12.values[0]
print(f'Точно также, фильмы жанра "{data12.index[0]}" чаще всего ({gnr12}) становятся прибыльными.')

Точно также, фильмы жанра "Drama" чаще всего (560) становятся прибыльными.


### 13. У какого режиссера самые большие суммарные кассовые сборы?

In [32]:
# так как режиссеров у некоторых фильмов несколько - нужно преобразовать поле director 
# данные типа str преобразуем в спиоск методом .split с лямбда-функцией
data13= data.copy()
data13.director = data13.director.apply(lambda dr: str(dr).split('|'))

# клонируем строки, у которых в списке несколько режиссеров - теперь у нас нужный датасет 
data13 = data13.explode('director')

# группируем датасет по полю director, суммируя поле revenue, сортируем по убыванию.
data13 = data13.groupby('director').revenue.sum().sort_values(ascending=False)

# теперь первый индекс сортировки - наш ответ
answeres['13'] = data13.index[0]

In [33]:
# information about answer
print(f'Самый прибыльный в Голливуде - это {data13.index[0]}, чьи фильмы принесли {data13.values[0]:,d} кассовых сборов.')

Самый прибыльный в Голливуде - это Peter Jackson, чьи фильмы принесли 6,490,593,685 кассовых сборов.


### 14. Какой режисер снял больше всего фильмов в стиле Action?

In [34]:
# так как жанров у некоторых фильмов несколько - используем функцию .str.contains() для фильтрации
data14 = data.copy()
data14 = data14[data14.genres.str.contains('Action', na=False)]

# клонируем строки, у которых в списке несколько режиссеров
data14.director = data14.director.apply(lambda dr: str(dr).split('|'))
data14 = data14.director.explode().value_counts()

# теперь считаем и сортируем ряд с режиссерами - первое значение наш ответ
answeres['14'] = data14.index[0]

In [35]:
# information about answer
print(f'{data14.index[0]} снял больше всего ({data14.values[0]}) фильмов в жанре экшн.')

Robert Rodriguez снял больше всего (9) фильмов в жанре экшн.


### 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году?

In [36]:
# с начала отберем фильмы за 2012 год
data15 = data[data.release_year == 2012].copy()

# так как актеров в каждом филме несколько (точнее - пять), разделим их в список и клонируем строки
data15.cast = data15.cast.apply(lambda cst: str(cst).split('|'))
data15 = data15.explode('cast')

# теперь группируем строки по актерам, считаем сумму кассовых сборов, сортируем по убыванию
data15 = data15.groupby('cast').revenue.sum().sort_values(ascending=False)

# наш ответ - первый в ряду
answeres['15'] = data15.index[0]

In [37]:
# information about answer
print(f'Кто такой {data15.index[0]}? А это актер, который снялся в фильмах, набравших наибольшую сумму в прокате!')

Кто такой Chris Hemsworth? А это актер, который снялся в фильмах, набравших наибольшую сумму в прокате!


### 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [38]:
# с начала отберем фильмы с бюджетом, выше средненго = высокобюджетные
data16 = data[data.budget > data.budget.mean()].copy()

# точно также как и ранее, разделим их в список и клонируем строки, а также сразу и посчитаем
data16 = data16.cast.apply(lambda cst: str(cst).split('|'))
data16 = data16.explode('cast').value_counts()

# наш ответ - первый в ряду
answeres['16'] = data16.index[0]

In [39]:
data_16 = data[data.cast.str.contains(data16.index[0], na=False)].copy()
data_16 = data_16.budget.sum()
data_16

print(f'На какого актера не жалко тратить столько денег: {data_16:,d} ???  Правильно: {data16.index[0]} !')

На какого актера не жалко тратить столько денег: 1,739,200,000 ???  Правильно: Matt Damon !


### 17. В фильмах какого жанра больше всего снимался Nicolas Cage?

In [40]:
# выберем все фильмы, где снялся Николас Кейдж, поле Жанр разделим на список жанров, клонируем строки и считаем...
data17 = data[data.cast.str.contains('Nicolas Cage', na=False)].copy()
data17 = data17.genres.str.split('|').explode().value_counts()

# наш ответ - первый в ряду
answeres['17'] = data17.index[0]

In [41]:
f'Казалось бы, в кам же жанре чаще всего снимается Николас Кейдж? Сюрприза не будет - {data17.index[0]}!'

'Казалось бы, в кам же жанре чаще всего снимается Николас Кейдж? Сюрприза не будет - Action!'

### 18. Самый убыточный фильм от Paramount Pictures

In [42]:
# version 1
# Выберем все фильмы этой кинокомпании, затем - найдем запись с самым убыточным фильмом
data_18 = data[(data.production_companies.str.contains('Paramount Pictures', na=False))]
data_18 = data_18[data_18.profit == data_18.profit.min()]

# теперь получим его название
answeres['18'] = data_18.original_title.describe().top

In [43]:
# version 2
# Выберем все фильмы этой кинокомпании
data18 = data[(data.production_companies.str.contains('Paramount Pictures', na=False))]

# затем - сортируем по возрастанию суммы. Первый в списке - искомый фильм
answeres['18'] = data18.sort_values('profit').iloc[0].original_title  

In [44]:
# information about answer
ds18 = data_18.original_title.describe().top
ls18 = data_18.profit.min()
print(f'У компании Paramount Pictures "{ds18}", оказался самым убыточным: {ls18:,d}')

У компании Paramount Pictures "K-19: The Widowmaker", оказался самым убыточным: -64,831,034


### 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [45]:
# просто сгруппируем фильмы по годам и сложим сумму кассовых сборов, сортируем по убыванию и берем первый в списке
answeres['19'] = data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False).index[0]

In [46]:
# information about answer
year19 = data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False).index[0]
revenue19 = data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False).values[0]
print(f'Самым успешным для Голливуда стал {year19} год с кассовыми сборами {revenue19:,d}')

Самым успешным для Голливуда стал 2015 год с кассовыми сборами 25,449,202,382


### 20. Какой самый прибыльный год для студии Warner Bros?

In [47]:
# отфильтруем фильмы студии
data_20 = data[(data.production_companies.str.contains('Warner Bros', na=False))]

# сгруппируем по годам и сложим прибыль/убытки, сортирум по убыванию, берм первый в ряду 
answeres['20'] = data_20.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).index[0]

In [48]:
# information about answer
year20 = data_20.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).index[0]
sum20 = data_20.groupby(['release_year'])['profit'].sum().max()
print(f'Самым прибыльным для студии Warner Bros. стал {year20} год, когда было заработано {sum20}')

Самым прибыльным для студии Warner Bros. стал 2014 год, когда было заработано 2295464519


### 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [49]:
# учитывая предобработку, где я выделил из даты отдельно месяц,
# просто сгруппируем фильмы по месяцам, посчитаем, отсортируем по убыванию, возьмем первый в ряду
# а еще применим атрибут из модуля calender, чтобы получить название месяца из его номера
answeres['21'] = clnd.month_name[data.groupby(['release_month'])['imdb_id']
                                 .count()
                                 .sort_values(ascending=False)
                                 .index[0]]

In [50]:
# information about answer
month21 = answeres['21']
a21 = data.groupby(['release_month'])['imdb_id'].count().sort_values(ascending=False).values[0]
print(f'Больше всего ({a21}) фильмов вышло в прокат в {month21}.')

Больше всего (227) фильмов вышло в прокат в September.


### 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [51]:
# учитывая предобработку, где я выделил из даты отдельно месяц, по которому вычислил время года,
# просто сгруппируем фильмы времени года = лето и посчитаем
answeres['22'] = data[data.year_time == "Summer"]['imdb_id'].count()

In [52]:
# information about answer
a22 = data[data.year_time == "Summer"]['imdb_id'].count()
print(f'Летом в прокат вышло суммарно {a22} фильмов.')

Летом в прокат вышло суммарно 450 фильмов.


### 23. Для какого режиссера зима – самое продуктивное время года?

In [53]:
# учитывая предобработку, где я выделил из даты отдельно месяц, по которому вычислил время года,
# просто сгруппируем фильмы времени года = зима
data23 = data[data.year_time == "Winter"].copy()

# клонируем строки, у которых в списке несколько режиссеров
data23.director = data23.director.apply(lambda dr: str(dr).split('|'))
data23 = data23.explode('director')

# сгруппируем по режиссерам, посчитаем, сортируем по убыванию
data23 = data23.groupby('director').imdb_id.count().sort_values(ascending=False)

# первый в ряду - наш ответ
answeres['23'] = data23.index[0]

In [54]:
# information about answer
print(f'Зимой больше всех ({data23.values[0]}) фильмов в прокат выпустил {data23.index[0]}.')

Зимой больше всех (7) фильмов в прокат выпустил Peter Jackson.


### 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [55]:
# скопируем датасет и добавим в него новое поле, где посчитаем длинну названия фильма
data24 = data.copy()
data24['title_len'] = data24.original_title.apply(lambda ttl: len(ttl))

# Клонируем строки для разделения поля "студия"
data24.production_companies = data24.production_companies.apply(lambda cst: str(cst).split('|'))
data24 = data24.explode('production_companies')

# групируем строки и получаем среднее значение длинны названия фильмов каждой студии, сортируем по убыванию
data24 = data24.groupby('production_companies').title_len.mean().sort_values(ascending=False)

# наш ответ - первый в ряду
answeres['24'] = data24.index[0]

In [56]:
# information about answer
print(f'Самое длинное название фильма составляет в длинну {data24.values[0]} симовлов.')
print(f'Оно же и среднее, так как студия {data24.index[0]}, выпустившяя его - сняла только один фильм')

Самое длинное название фильма составляет в длинну 83.0 симовлов.
Оно же и среднее, так как студия Four By Two Productions, выпустившяя его - сняла только один фильм


### 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [57]:
data25 = pd.read_csv('C:\Projects_0\input_data\movie_bd_v5.xls').copy()

# в начале разделим поле описания на список из слов
data25.overview = data25.overview.apply(lambda s: str(s).split(' '))

# добавим в датасет новое поле, где посчитаем количество слов в описании
# для этого используем функцию Counter из модуля collections
data25['overview_len'] = data25.overview.apply(lambda o: sum(Counter(o).values()))

# клонируем строки для разделения поля "студия" 
data25.production_companies = data25.production_companies.apply(lambda cst: str(cst).split('|'))
data25 = data25.explode('production_companies')

# группируем по полю студия, считаем среднее значение количества слов, сортируем по убыванию
data25 = data25.groupby('production_companies').overview_len.mean().sort_values(ascending=False)

# наш ответ - первый в ряду
answeres['25'] = data25.index[0]

In [58]:
# information about answer
print(f'Самые словоохотливые оказались в {data25.index[0]}, придумав описание своего творения ')
print(f'аж на {data25.values[0]} слов. И конечно, он у них единственный...')

Самые словоохотливые оказались в Midnight Picture Show, придумав описание своего творения 
аж на 175.0 слов. И конечно, он у них единственный...


### 26. Какие фильмы входят в 1 процент лучших по рейтингу?

In [59]:
# просто сортируем датасет по полю "рейтинг" по убыванию и возьмем первые 19 строк
data26 = data.copy()
data26 = data26.sort_values('vote_average', ascending=False).iloc[0:19]

# для ответа преобразуем ряд в кортеж
answeres['26'] = tuple(data26.original_title.unique())

In [60]:
# information about answer
data26[['original_title','vote_average']]

,original_title,vote_average
599,The Dark Knight,8.1
118,Interstellar,8.0
125,The Imitation Game,8.0
9,Inside Out,8.0
34,Room,8.0
1183,The Wolf of Wall Street,7.9
128,Gone Girl,7.9
1191,12 Years a Slave,7.9
119,Guardians of the Galaxy,7.9
1081,The Lord of the Rings: The Return of the King,7.9


### 27. Какие актеры чаще всего снимаются в одном фильме вместе?

In [61]:
data27 = data.copy()

# с начала разделим строку в поле cast на список имен актеров
data27.cast = data27.cast.apply(lambda cst: str(cst).split('|'))

# с помощью функции перестановок из itertools составим в поле cast пары из актеров в виде кортежей
# чтобы не получить "зеркальные варианты", в каждом поле с начала отсоритруем имена по возрастанию
data27.cast = data27.cast.apply(lambda cst: tuple(it.combinations(sorted(cst),2)))

# клонируем по полю cast
data27 = data27.explode('cast')

# группируем по полю cast, считаем фильмы, сортируем по убыванию
data_27 = data27.groupby('cast').imdb_id.count().sort_values(ascending=False)

# собираем записи, где число совместных фильмов максимальное
data_27 = data_27[data_27.values == data_27.values[0]]

# для ответа преобразуем ряд в кортеж
answeres['27'] = tuple(data_27.index)

In [62]:
# information about answer
pair1 = data_27.index[0]    # первая пара из ответ
films27 = data_27.values[0] # количество фильмов, где пары снялись вместе

print(f'Вот список пар актеров снявшихся совместно в {films27} фильмах: ]')
print(tuple(data_27.index))
print()
print(f'Но "ларчик открывается просто", когда мы взглянем на эти фильмы:')
tuple(data27[(data27.cast == pair1)].original_title)

Вот список пар актеров снявшихся совместно в 8 фильмах: ]
(('Daniel Radcliffe', 'Rupert Grint'), ('Emma Watson', 'Rupert Grint'), ('Daniel Radcliffe', 'Emma Watson'))

Но "ларчик открывается просто", когда мы взглянем на эти фильмы:


('Harry Potter and the Half-Blood Prince',
 'Harry Potter and the Deathly Hallows: Part 1',
 "Harry Potter and the Philosopher's Stone",
 'Harry Potter and the Deathly Hallows: Part 2',
 'Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Prisoner of Azkaban',
 'Harry Potter and the Order of the Phoenix')

# Submission

In [63]:
answeres

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': 109.6585494970884,
 '5': 107.0,
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': 1478,
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 'September',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ('The Dark Knight',
  'Interstellar',
  'The Imitation Game',
  'Inside Out',
  'Room',
  'The Wolf of Wall Street',
  'Gone Girl',
  '12 Years a Slave',
  'Guardians of the Galaxy',
  'The Lord of the Rings: The Return of the King',
  'Memento',
  'Inception',
  'The Pianist',
  'The Grand Budapest Hotel',
  'Her',
  'Spotlight',
  'Big Hero 6',
  'The Fault in Our Stars',
  'The Lord of the Rings: The Two Towers'),

In [64]:
len(answeres)

27